In [1]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
%%time
inter_df = pd.read_csv('floor_info_maindata_inter_baseline_2.csv')

Wall time: 21.1 s


In [3]:
inter_df.shape

(6880841, 16)

In [4]:
inter_df.head()

,관리_건축물대장_PK,건물_명,층_구분,층_번호,층_번호_명,구조,기타_구조,주_용도,기타_용도,면적(㎡),주_부속_구분,면적_제외_여부,생성_일자,기준년월,지번주소,addr_split_len
0,11440-3279202,중동빌라,지상,1,1층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,57.62,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
1,11440-3279202,중동빌라,지상,2,2층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
2,11440-3279202,중동빌라,지상,3,3층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
3,11440-3279202,중동빌라,지상,4,4층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,59.68,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
4,11440-3279202,중동빌라,옥탑,1,옥탑 1층,철근콘크리트구조,철근콘크리트구조,다세대주택,계단실(연면적제외),7.68,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4


In [5]:
hannam_df = inter_df[inter_df['지번주소'].str.contains('한남동 810')]
print(hannam_df.shape)

(1864, 16)


In [6]:
hannam_df.head(100)

,관리_건축물대장_PK,건물_명,층_구분,층_번호,층_번호_명,구조,기타_구조,주_용도,기타_용도,면적(㎡),주_부속_구분,면적_제외_여부,생성_일자,기준년월,지번주소,addr_split_len
319800,11170-10018927302,한남더힐,지상,3,3층,철근콘크리트구조,철근콘크리트구조,아파트,아파트,351.592,주건축물,0.0,20141109,201412,서울특별시 용산구 한남동 810-0,4
319801,11170-10018927302,한남더힐,지상,5,5층,철근콘크리트구조,철근콘크리트구조,아파트,아파트,351.592,주건축물,0.0,20141109,201412,서울특별시 용산구 한남동 810-0,4
319802,11170-10018927302,한남더힐,지상,10,10층,철근콘크리트구조,철근콘크리트구조,아파트,아파트,351.592,주건축물,0.0,20141109,201412,서울특별시 용산구 한남동 810-0,4
319803,11170-10018927302,한남더힐,지상,2,2층,철근콘크리트구조,철근콘크리트구조,아파트,아파트,351.592,주건축물,0.0,20141109,201412,서울특별시 용산구 한남동 810-0,4
319804,11170-10018927302,한남더힐,지상,1,1층,철근콘크리트구조,철근콘크리트구조,아파트,아파트,351.592,주건축물,0.0,20141109,201412,서울특별시 용산구 한남동 810-0,4
319805,11170-10018927302,한남더힐,지상,6,6층,철근콘크리트구조,철근콘크리트구조,아파트,아파트,351.592,주건축물,0.0,20141109,201412,서울특별시 용산구 한남동 810-0,4
319806,11170-10018927302,한남더힐,지상,4,4층,철근콘크리트구조,철근콘크리트구조,아파트,아파트,351.592,주건축물,0.0,20141109,201412,서울특별시 용산구 한남동 810-0,4
319811,11170-10018927302,한남더힐,지상,8,8층,철근콘크리트구조,철근콘크리트구조,아파트,아파트,351.592,주건축물,0.0,20141109,201412,서울특별시 용산구 한남동 810-0,4
319812,11170-10018927302,한남더힐,지상,7,7층,철근콘크리트구조,철근콘크리트구조,아파트,아파트,351.592,주건축물,0.0,20141109,201412,서울특별시 용산구 한남동 810-0,4
319813,11170-10018927302,한남더힐,지하,1,지1층,철근콘크리트구조,철근콘크리트구조,아파트,아파트,173.889,주건축물,0.0,20141109,201412,서울특별시 용산구 한남동 810-0,4


In [7]:
inter_df['주_부속_구분'].value_counts()

주건축물     6721119
부속건축물     159241
Name: 주_부속_구분, dtype: int64